# 3. Train-Predict LSTM

## Result:
- Kaggle score: 

## Tensorboard
- Input at command: tensorboard --logdir=./log
- Input at browser: http://127.0.0.1:6006

## Reference
- https://www.kaggle.com/codename007/a-very-extensive-landmark-exploratory-analysis

## Run name

In [1]:
import time
import os
import pandas as pd
import gc
import sys

project_name = 'Google_LandMark_Rec'
step_name = 'LSTM_Bidirectional'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: Google_LandMark_Rec_LSTM_Bidirectional_20180328_163937


## 项目文件夹

In [2]:
cwd = os.getcwd()
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
feature_folder = os.path.join(cwd, 'feature')
post_pca_feature_folder = os.path.join(cwd, 'post_pca_feature')
log_folder = os.path.join(cwd, 'log')
print('input_folder: \t\t\t' + input_folder)
print('output_folder: \t\t\t' + output_folder)
print('model_folder: \t\t\t' + model_folder)
print('feature_folder: \t\t' + feature_folder)
print('post_pca_feature_folder: \t' + post_pca_feature_folder)
print('log_folder: \t\t\t' + log_folder)

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

if not os.path.exists(post_pca_feature_folder):
    os.mkdir(post_pca_feature_folder)
    print('Create folder: %s' % post_pca_feature_folder)

input_folder: 			/data1/kaggle/landmark-recognition-challenge/input
output_folder: 			/data1/kaggle/landmark-recognition-challenge/output
model_folder: 			/data1/kaggle/landmark-recognition-challenge/model
feature_folder: 		/data1/kaggle/landmark-recognition-challenge/feature
post_pca_feature_folder: 	/data1/kaggle/landmark-recognition-challenge/post_pca_feature
log_folder: 			/data1/kaggle/landmark-recognition-challenge/log


In [3]:
train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## 获取landmark_id集合

In [4]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(10))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(10))

train_csv.shape is (1225029, 3).


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651
2,6b2bb500b6a38aa0,http://lh6.ggpht.com/-vKr5G5MEusk/SR6r6SJi6mI/...,11284
3,b399f09dee9c3c67,https://lh3.googleusercontent.com/-LOW2cjAqubA...,8429
4,19ace29d77a5be66,https://lh5.googleusercontent.com/-tnmSXwQcWL8...,6231
5,2c9c54b62f0a6a37,https://lh5.googleusercontent.com/-mEaSECO7D-4...,10400
6,0aac70a1de44ced6,http://lh6.ggpht.com/-cJMh9AYQGk8/SOkF_Q5PrjI/...,9779
7,de770bc720f8e714,https://lh4.googleusercontent.com/-Q_FvRlwaaa8...,11288
8,dc9457d703e612ad,https://lh3.googleusercontent.com/-Px33Q-wekRI...,13170
9,3060f5f75d936abb,http://lh3.ggpht.com/-KXyELwqwp_Q/Ry-qmQAqwUI/...,6051


test_csv.shape is (117703, 2).


,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...
2,0001bbb682d45002,https://lh3.googleusercontent.com/-kloLenz1xZk...
3,0002362830cfe3a3,https://lh3.googleusercontent.com/-N6z79jNZYTg...
4,000270c9100de789,https://lh3.googleusercontent.com/-keriHaVOq1U...
5,0002b0fab5d3ccc4,https://lh3.googleusercontent.com/-ciWklpsrab8...
6,000396be3c24830a,https://lh3.googleusercontent.com/-6W9F179t59Q...
7,000506dc6ab3a40e,https://lh3.googleusercontent.com/-_XHsAXB2LZA...
8,0005292fc4b005a3,https://lh3.googleusercontent.com/-RBZ4F1ZKNc0...
9,0005456a82264bc8,https://lh3.googleusercontent.com/-MRK7_uiKO6A...


In [5]:
train_id = train_csv['id']
train_landmark_id = train_csv['landmark_id']
unique_landmark_ids = list(set(train_landmark_id))
len_unique_landmark_ids = len(unique_landmark_ids)
print(unique_landmark_ids[:10]) # 确认landmark_id是从0开始
print('len(unique_landmark_ids)=%d' % len_unique_landmark_ids)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len(unique_landmark_ids)=14951


## 预览sample_submission.csv

In [6]:
sample_submission_csv = pd.read_csv(sample_submission_folder)
print('sample_submission_csv.shape is {0}.'.format(sample_submission_csv.shape))
display(sample_submission_csv.head(2))

sample_submission_csv.shape is (117703, 2).


,id,landmarks
0,000088da12d664db,8815 0.03
1,0001623c6d808702,7249 0.61


## 加载feature

In [7]:
%%time
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(2018)


def load_post_pac_feature(model_name, n_components, image_size, time_str, post_pca_feature_folder):
    x_data = {}
    y_data = {}
    x_test = {}
    
    feature_model = os.path.join(post_pca_feature_folder, 'post_pca_feature_%s_%s_%s_%s.h5' % (model_name, n_components, image_size, time_str))
    for filename in [feature_model]:
        with h5py.File(filename, 'r') as h:
            x_data = np.array(h['train'])
            y_data = np.array(h['train_labels'])
            x_test = np.array(h['test'])
    return x_data, y_data, x_test

def load_feature(model_name, image_size, time_str, feature_folder):
    x_data = {}
    y_data = {}
    x_test = {}
    
    feature_model = os.path.join(feature_folder, 'feature_%s_%s_%s.h5' % (model_name, image_size, time_str))
    for filename in [feature_model]:
        with h5py.File(filename, 'r') as h:
            x_data = np.array(h['train'])
            y_data = np.array(h['train_labels'])
            x_test = np.array(h['test'])
    return x_data, y_data, x_test

CPU times: user 124 ms, sys: 8 ms, total: 132 ms
Wall time: 186 ms


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
%%time
x_data_Xception, y_data_Xception, x_test_Xception = load_feature('Xception', 200, '20180312-050926', feature_folder)
print('*' * 60)
print(x_data_Xception.shape)
print(len(y_data_Xception))
print(x_test_Xception.shape)

x_data_InceptionV3, y_data_InceptionV3, x_test_InceptionV3 = load_feature('InceptionV3', 200, '20180312-050926', feature_folder)
print('*' * 60)
print(x_data_InceptionV3.shape)
print(len(y_data_InceptionV3))
print(x_test_InceptionV3.shape)

# x_data_InceptionResNetV2, y_data_InceptionResNetV2, x_test_InceptionResNetV2 = load_feature('InceptionResNetV2', 200, '20180312-050926', feature_folder)
# print('*' * 60)
# print(x_data_InceptionResNetV2.shape)
# print(len(y_data_InceptionResNetV2))
# print(x_test_InceptionResNetV2.shape)

************************************************************
(1219426, 2048)
1219426
(115942, 2048)
************************************************************
(1219426, 2048)
1219426
(115942, 2048)
CPU times: user 6.52 s, sys: 25.5 s, total: 32 s
Wall time: 3min 10s


In [9]:
import sys
def describe(arr):
    print(arr.shape, arr.min(), arr.max(), sys.getsizeof(arr))

In [10]:
%%time
x_data = np.concatenate([x_data_Xception[:, np.newaxis, :],
                         x_data_InceptionV3[:, np.newaxis, :]], axis=1)
describe(x_data)
del x_data_Xception
del x_data_InceptionV3
# del band_avg_data
gc.collect()

x_test = np.concatenate([x_test_Xception[:, np.newaxis, :],
                         x_test_InceptionV3[:, np.newaxis, :]], axis=1)
describe(x_test)
del x_test_Xception
del x_test_InceptionV3
# del band_avg_test
gc.collect()

y_data = y_data_Xception
describe(y_data)

(1219426, 2, 2048) 0.0 22.508612 19979075712
(115942, 2, 2048) 0.0 17.827248 1899593856
(1219426,) 0 14950 4877800
CPU times: user 8.98 s, sys: 9.01 s, total: 18 s
Wall time: 18 s


In [11]:
%%time
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.01, random_state=5)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(1207231, 2, 2048)
(12195, 2, 2048)
(1207231,)
(12195,)
CPU times: user 3.98 s, sys: 7.42 s, total: 11.4 s
Wall time: 11.4 s


In [12]:
from keras.utils.np_utils import to_categorical

def data_generator(x_train, y_train, batch_size, num_classes):
    len_x_train = len(x_train)
    start_index = 0
    end_index = 0
    while(1):
        end_index = start_index + batch_size
        if end_index < len_x_train:
#             print(start_index, end_index, end=' ')
            x_batch = x_train[start_index: end_index, :]
            y_batch = y_train[start_index: end_index]
            y_batch_cat = to_categorical(y_batch, num_classes)
            
            start_index = start_index + batch_size
#             print(x_batch.shape, y_batch_cat.shape)
            yield x_batch, y_batch_cat
        else:
            end_index = end_index-len_x_train
#             print(start_index, end_index, end=' ')
            x_batch = np.vstack((x_train[start_index:, :], x_train[:end_index, :]))
            y_batch = np.concatenate([y_train[start_index:], y_train[:end_index]])
            y_batch_cat = to_categorical(y_batch, num_classes)
            
            start_index = end_index
#             print(x_batch.shape, y_batch_cat.shape)
            yield x_batch, y_batch_cat

Using TensorFlow backend.


In [13]:
data_dim = x_data.shape[2]
timesteps = x_data.shape[1]
num_classes = len(list(set(y_data)))
print(data_dim, timesteps, num_classes)
print('data_dim: \t%s' % data_dim)
print('timesteps: \t%s' % timesteps)
print('num_classes: \t%s' % num_classes)


print('*' * 60)
len_train = len(y_train)
batch_size = 2048
steps_per_epoch_train = int(len_train/batch_size)
print('train data: %s %s %s' % (len_train, batch_size, steps_per_epoch_train))

train_gen = data_generator(x_train, y_train, batch_size, num_classes)
batch_data = next(train_gen)
print(batch_data[0].shape, batch_data[1].shape)


print('*' * 60)
len_val = len(y_val)
steps_per_epoch_val = int(len_val/batch_size)
print('val data: %s %s %s' % (len_val, batch_size, steps_per_epoch_val))

val_gen = data_generator(x_val, y_val, batch_size, num_classes)
batch_data = next(val_gen)
print(batch_data[0].shape, batch_data[1].shape)

2048 2 14951
data_dim: 	2048
timesteps: 	2
num_classes: 	14951
************************************************************
train data: 1207231 2048 589
(2048, 2, 2048) (2048, 14951)
************************************************************
val data: 12195 2048 5
(2048, 2, 2048) (2048, 14951)


## Build NN

In [14]:
from sklearn.metrics import confusion_matrix

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LSTM, Bidirectional
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

In [15]:
def get_lr(x):
    lr = round(3e-4 * 0.98 ** x, 6)
    if lr < 1e-5:
        lr = 1e-5
    print(lr, end='  ')
    return lr

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)

log_dir = os.path.join(log_folder, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:/data1/kaggle/landmark-recognition-challenge/log/Google_LandMark_Rec_LSTM_Bidirectional_20180328_163937


In [16]:
# expected input data shape: (batch_size, timesteps, data_dim)
model = Sequential()
model.add(Bidirectional(LSTM(512, return_sequences=True), input_shape=(timesteps, data_dim)))  # returns a sequence of vectors of dimension 32
model.add(Dropout(0.3))
# model.add(Bidirectional(LSTM(512, return_sequences=True)))  # returns a sequence of vectors of dimension 32
# model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(512)))  # return a single vector of dimension 32
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer=Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 2, 1024)           10489856  
_________________________________________________________________
dropout_1 (Dropout)          (None, 2, 1024)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1024)              6295552   
_________________________________________________________________
dense_1 (Dense)              (None, 14951)             15324775  
Total params: 32,110,183
Trainable params: 32,110,183
Non-trainable params: 0
_________________________________________________________________


In [ ]:
%%time
hist = model.fit_generator(train_gen,
    steps_per_epoch=steps_per_epoch_train,
    epochs=20, #Increase this when not on Kaggle kernel
    verbose=1,  #1 for ETA, 0 for silent
    callbacks=[annealer],
    max_queue_size=2,
    workers=4,
    use_multiprocessing=False,
    validation_data=val_gen,
    validation_steps=steps_per_epoch_val)

Epoch 1/20
 17/589 [..............................] - ETA: 1:59:30 - loss: 8.7378 - acc: 0.0512

In [ ]:
final_loss, final_acc = model.evaluate_generator(val_gen, steps=steps_per_epoch_val)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
run_name_acc = run_name + '_' + str(int(final_acc*10000)).zfill(4)

In [ ]:
histories = pd.DataFrame(hist.history)
histories['epoch'] = hist.epoch
print(histories.columns)
histories_file = os.path.join(model_folder, run_name_acc + '.csv')
histories.to_csv(histories_file, index=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(hist.history['loss'], color='b')
plt.plot(hist.history['val_loss'], color='r')
plt.show()
plt.plot(hist.history['acc'], color='b')
plt.plot(hist.history['val_acc'], color='r')
plt.show()

In [ ]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
saveModel(model, run_name_acc)

## Predict

In [ ]:
y_pred = model.predict(x_test, batch_size=128)
print(y_pred.shape)

In [ ]:
# 这里证明os.listdir()得到的图片名称list不正确
files = os.listdir(os.path.join(cwd, 'input', 'data_test', 'test'))
print(files[:10])

In [ ]:
# 这里证明ImageDataGenerator()得到的图片名称list才是正确
gen = ImageDataGenerator()
image_size = (299, 299)
batch_size = 128
test_generator  = gen.flow_from_directory(test_folder, image_size, shuffle=False, batch_size=batch_size)
print('test_generator')
print(len(test_generator.filenames))
print(test_generator.filenames[:10])

In [ ]:
%%time
max_indexes = np.argmax(y_pred, -1)
print(max_indexes.shape)

test_dict = {}
for i, paire in enumerate(zip(test_generator.filenames, max_indexes)):
    image_name, indx = paire[0], paire[1]
    image_id = image_name[5:-4]
#     test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])
    test_dict[image_id] = '%d %.4f' % (indx, y_pred[i, indx])

#确认图片的id是否能与ImageDataGenerator()对应上
for key in list(test_dict.keys())[:10]:
    print('%s  %s' % (key, test_dict[key]))

In [ ]:
display(sample_submission_csv.head(2))

In [ ]:
%%time
len_sample_submission_csv = len(sample_submission_csv)
print('len(len_sample_submission_csv)=%d' % len_sample_submission_csv)
count = 0
for i in range(len_sample_submission_csv):
    image_id = sample_submission_csv.iloc[i, 0]
#     landmarks = sample_submission_csv.iloc[i, 1]
    if image_id in test_dict:
        pred_landmarks = test_dict[image_id]
#         print('%s  %s' % (image_id, pred_landmarks))
        sample_submission_csv.iloc[i, 1] = pred_landmarks
    else:
#         print(image_id)
#         sample_submission_csv.iloc[i, 1] = '9633 1.0' # 属于9633的类最多，所以全都设置成这个类，可能会比设置成空得到的结果好
        sample_submission_csv.iloc[i, 1] = '' # 设置成空
    count += 1
    if count % 10000 == 0:
        print(int(count/10000), end=' ')
display(sample_submission_csv.head(2))

In [ ]:
pred_file = os.path.join(output_folder, 'pred_' + run_name_acc + '.csv')
sample_submission_csv.to_csv(pred_file, index=None)

In [ ]:
print(run_name_acc)
print('Done !')